In [ ]:
# dependencies
# !pip install torch numpy transformers datasets tiktoken wandb tqdm
# !pip install tiktoken
# !pip install sentencepiece

In [1]:
# imports
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
from typing import Tuple, Callable, Dict, List, Optional

In [2]:
def load_data(input_path: str) -> Tuple[str, int]:
    # Loading training text data
    with open(input_path, "r", encoding='utf-8') as f:
        text = f.read()
    print(f'Total number of characters = {len(text)}')
    chars = sorted(list(set(text)))
    vocab_size = len(chars)
    print(f'Total number of unique characters = {vocab_size}')
    print(f"Characters = {''.join(chars)}")
    return text, vocab_size

In [3]:
input_path = 'input.txt'
text, vocab_size = load_data(input_path)

Total number of characters = 1115394
Total number of unique characters = 65
Characters = 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [4]:
def choose_tokenizer(tokenizer_type: str) -> Tuple[Callable, Callable, int]:
    '''choose_tokenizer should return a tuple of two callable objects: tokenizer and decoder'''
    test_phrase = 'Hello Lord!'
    if tokenizer_type == 'simple':
        chars = sorted(list(set(text)))
        vocab_size = len(chars)
        stoi = {ch: i for i, ch in enumerate(chars)}
        itos = {i: ch for i, ch in enumerate(chars)}
        encoder = lambda x: [stoi[ch] for ch in x]
        decoder = lambda x: ''.join([itos[i] for i in x])
        print(f'Number of vocabulary = {vocab_size}')
        print(f'Encoding {test_phrase}: {encoder(test_phrase)}')
        print(f'Decoding {test_phrase}: {decoder(encoder(test_phrase))}')
    elif tokenizer_type == 'sentencepiece':
        import sentencepiece as spm
        vocab_size = 1000
        params = (''.join(['--input=', input_path, ' ', '--model_prefix=spm ', '--vocab_size=', str(vocab_size), ' ']))
        spm.SentencePieceTrainer.Train(params)
        sp = spm.SentencePieceProcessor()
        sp.Load('spm.model')
        encoder = sp.EncodeAsIds
        decoder = sp.DecodeIds
        print(f'Number of vocabulary = {vocab_size}')
        print(f"Encoding {test_phrase} as pieces: {sp.EncodeAsPieces(test_phrase)}")
        print(f"Encoding {test_phrase} as IDs: {encoder(test_phrase)}")
        print(f"Decoding IDs: {decoder(encoder(test_phrase))}")
    elif tokenizer_type == 'tiktoken':
        import tiktoken
        enc = tiktoken.get_encoding("cl100k_base")
        vocab_size = enc.n_vocab
        encoder = enc.encode
        decoder = enc.decode
        print(f'Number of vocabulary = {vocab_size}')
        print(f'Encoding {test_phrase}: {encoder(test_phrase)}')
        print(f'Decoding {test_phrase}: {decoder(encoder(test_phrase))}')

    return encoder, decoder, vocab_size

In [5]:
encoder1, decoder1, vocab_size1 = choose_tokenizer('simple')
encoder2, decoder2, vocab_size2 = choose_tokenizer('sentencepiece')
encoder3, decoder3, vocab_size3 = choose_tokenizer('tiktoken')

Number of vocabulary = 65
Encoding Hello Lord!: [20, 43, 50, 50, 53, 1, 24, 53, 56, 42, 2]
Decoding Hello Lord!: Hello Lord!
Number of vocabulary = 1000
Encoding Hello Lord! as pieces: ['▁He', 'll', 'o', '▁Lord', '!']
Encoding Hello Lord! as IDs: [184, 65, 25, 338, 32]
Decoding IDs: Hello Lord!
Number of vocabulary = 100277
Encoding Hello Lord!: [9906, 10425, 0]
Decoding Hello Lord!: Hello Lord!


In [6]:
data1 = torch.tensor(encoder1(text),dtype=torch.long)
data2 = torch.tensor(encoder2(text),dtype=torch.long)
data3 = torch.tensor(encoder3(text),dtype=torch.long)
print(f'Using simple tokenizer, there are {data1.shape[0]} tokens, \nUsing sentencepiece tokenizer, there are {data2.shape[0]} tokens, \nUsing tiktoken tokenizer, there are {data3.shape[0]} tokens.\n--------')
sample_length = 30
print(f'Simple tokenizer: {data1[:sample_length]}\n{decoder1(data1[:sample_length].tolist())}\n--------')
print(f'Sentencepiece tokenizer: {data2[:sample_length]}\n{decoder2(data2[:sample_length].tolist())}\n--------')
print(f'Tiktoken tokenizer: {data3[:sample_length]}\n{decoder3(data3[:sample_length].tolist())}\n--------')

Using simple tokenizer, there are 1115394 tokens, 
Using sentencepiece tokenizer, there are 407344 tokens, 
Using tiktoken tokenizer, there are 301829 tokens.
--------
Simple tokenizer: tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43])
First Citizen:
Before we proce
--------
Sentencepiece tokenizer: tensor([298, 537,   6, 259, 280,  12,  81, 231, 107,  35, 384, 987,   3, 223,
         33, 254,   7,  52,  65,   6,  78, 358,  21,  73,   3, 254,   7, 298,
        537,   6])
First Citizen: Before we proceed any further, hear me speak. All: Speak, speak. First Citizen:
--------
Tiktoken tokenizer: tensor([ 5451, 47317,   512, 10438,   584, 10570,   904,  4726,    11,  6865,
          757,  6604,   382,  2460,   512, 96945,    11,  6604,   382,  5451,
        47317,   512,  2675,   527,   682, 20250,  4856,   311,  2815,  1109])
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, spea

In [7]:
# train-test split
n = 0.8 # train ratio
train_data1 = data1[:int(n*len(data1))]
val_data1 = data1[int(n*len(data1)):]

train_data2 = data2[:int(n*len(data2))]
val_data2 = data2[int(n*len(data2)):]

train_data3 = data3[:int(n*len(data3))]
val_data3 = data3[int(n*len(data3)):]

In [8]:
torch.manual_seed(1337)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
def get_batch(split: str, tokenizer: str, batch_size: int = 4, block_size: int = 8) -> Tuple[torch.Tensor, torch.Tensor]:
    if tokenizer == 'simple':
        if split == 'train':
            data = train_data1
        else:
            data = val_data1
    elif tokenizer == 'sentencepiece':
        if split == 'train':
            data = train_data2
        else:
            data = val_data2
    else:
        if split == 'train':
            data = train_data3
        else:
            data = val_data3
    idx = torch.randint(0, data.size(0) - block_size, (batch_size,)) # select random starting indices to form a batch
    x = torch.stack([data[i:i+block_size] for i in idx])
    y = torch.stack([data[i+1:i+block_size+1] for i in idx])
    x = x.to(device)
    y = y.to(device)
    return x, y

In [10]:
def show_sample(xb: torch.Tensor, yb: torch.Tensor, encoder: Callable, decoder: Callable, block_size: int = 8, decode: bool = False) -> None:
    # for each batch, there are 8 tokens in the input and target, but actually this contains 8 examples, corresponding to 8 different time steps, each with a target token that is shifted one position to the right
    for b in range(xb.shape[0]): # batch dimension
        for t in range(block_size): # time dimension
            context = xb[b, :t+1]
            target = yb[b,t]
            print(f"when input is {context.tolist()} the target: {target}")
            if decode == True:
                print(f'context = {decoder(context.tolist())} \ntarget = {decoder([target.tolist()])}')
        print('\n--------')

In [11]:
batch_size = 4
block_size = 8
xb1, yb1 = get_batch(split = 'train', tokenizer = 'simple')
xb2, yb2 = get_batch(split = 'train', tokenizer = 'sentencepiece')
xb3, yb3 = get_batch(split = 'train', tokenizer = 'tiktoken')

print(f'input shape = {xb1.shape}')
print(xb1) # input to the transformer
print(f'target shape = {yb1.shape}')
print(yb1)

print(f'input shape = {xb2.shape}')
print(xb2)
print(f'target shape = {yb2.shape}')
print(yb2)

print(f'input shape = {xb3.shape}')
print(xb3)
print(f'target shape = {yb3.shape}')
print(yb3)

print('----')
show_sample(xb1, yb1, encoder1, decoder1, block_size = 8, decode = False)
show_sample(xb2, yb2, encoder2, decoder2, block_size = 8, decode = False)
show_sample(xb3, yb3, encoder3, decoder3, block_size = 8, decode = False)

input shape = torch.Size([4, 8])
tensor([[58, 63,  8,  0,  0, 19, 24, 27],
        [39, 59, 45, 46, 58,  1, 46, 43],
        [49, 43, 57,  1, 53, 50, 42,  1],
        [52, 41, 47, 43, 52, 58,  1, 56]])
target shape = torch.Size([4, 8])
tensor([[63,  8,  0,  0, 19, 24, 27, 33],
        [59, 45, 46, 58,  1, 46, 43,  1],
        [43, 57,  1, 53, 50, 42,  1, 46],
        [41, 47, 43, 52, 58,  1, 56, 47]])
input shape = torch.Size([4, 8])
tensor([[ 29, 100,  68, 130,  31,   3, 442,  63],
        [  9,  56, 979,   8,   3, 247, 106, 490],
        [ 11,   8, 123, 923,  11,   5, 345,   3],
        [ 61, 127,   4, 132,  12, 284,  14, 352]])
target shape = torch.Size([4, 8])
tensor([[100,  68, 130,  31,   3, 442,  63, 959],
        [ 56, 979,   8,   3, 247, 106, 490,  27],
        [  8, 123, 923,  11,   5, 345,   3,  88],
        [127,   4, 132,  12, 284,  14, 352,  16]])
input shape = torch.Size([4, 8])
tensor([[ 1833,   499,   382,  5176,   785,   512,  1687,   527],
        [  584, 22525,   53

In [12]:
@torch.no_grad()
def estimate_loss(
    model: nn.Module, 
    eval_iters: int = 200, 
    tokenizer: str = 'simple', 
    batch_size: int = 32, 
    block_size: int = 8
) -> Dict[str, float]:
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xb, yb = get_batch(split, tokenizer = tokenizer, batch_size = batch_size, block_size = block_size)
            _, loss = model(xb, yb)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [13]:
class LayerNorm(nn.Module):
    def __init__(self, dim: int = 32, eps: float = 1e-5):
        super().__init__()
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        self.eps = eps
    def __call__(self, x: torch.Tensor) -> torch.Tensor:
        xmean =x.mean(1, keepdim=True) # layer mean, if dim = 0, then it is batch mean for batch norm
        xvar = x.var(1, keepdim=True) # layer variance, if dim = 0, then it is batch variance for batch norm
        xhat = (x-xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        return self.out
    def parameters(self):
        return [self.gamma, self.beta] # trainable parameters

class Head(nn.Module): # this is one attention head
    def __init__(self, head_size: int = 16, n_embd: int = 32, block_size: int = 8, dropout_r: float = 0.1):
        super().__init__()
        # hyperparameters
        self.head_size = head_size
        # layers
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout_r)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B,T,C = x.shape
        #print(f'x shape = {x.shape}')
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        wei = q @ k.transpose(-2, -1) * self.head_size**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        return wei @ v

# multi-head self-attention
class MultiHead(nn.Module):
    def __init__(self, head_size: int = 16, n_embd: int = 32, block_size: int = 8, n_heads: int = 8, dropout_r: float = 0.1):
        super().__init__()
        # layers
        self.heads = nn.ModuleList([Head(head_size, n_embd, 
                                         block_size, dropout_r) for _ in range(n_heads)])
        self.proj = nn.Linear(n_heads*head_size, n_embd)
        self.dropout = nn.Dropout(dropout_r)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = torch.cat([head(x) for head in self.heads], dim=-1)
        x = self.dropout(self.proj(x))
        return x

class FeedForward(nn.Module):
    def __init__(self, n_embd: int = 32, dropout_r: float = 0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout_r)
        )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)
    
class Block(nn.Module):
    def __init__(self, head_size: int = 16, n_embd: int = 32, block_size: int = 8, n_heads: int = 8, dropout_r: float = 0.1):
        super().__init__()
        self.multi_head = MultiHead(head_size, n_embd, block_size, n_heads, dropout_r)
        self.ff = FeedForward(n_embd, dropout_r)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.multi_head(self.ln1(x)) + x
        x = self.ff(self.ln2(x)) + x
        return x
        
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size: int = 65, n_embd: int = 32, block_size: int = 8, head_size: int = 16, n_heads: int = 8, n_blocks: int = 6, dropout_r: float = 0.1):
        super().__init__()
        self.block_size = block_size
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.positional_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            *[Block(n_embd = n_embd, block_size = block_size, head_size = head_size, n_heads = n_heads, dropout_r = dropout_r) for _ in range(n_blocks)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size) # language model head

    def forward(self, idx: torch.Tensor, targets: Optional[torch.Tensor] = None) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        # idx and targets are both (B,T) tensor of integers
        B, T = idx.shape
        pos_embd = self.positional_embedding_table(torch.arange(T, device=idx.device)) # (T,n_embd)
        # this is the embedded representation of the input, where idx is the input xb tensor
        tok_embd = self.token_embedding_table(idx) # (B,T,n_embd)
        embd = tok_embd + pos_embd # (B,T,n_embd), pytorch will broadcast pos_embd to (B,T,n_embd), C = n_embd
        embd = self.blocks(embd) # (B,T,n_embd)
        embd = self.ln_f(embd) # (B,T,n_embd)
        logits = self.lm_head(embd) # (B,T,C), C = vocab_size = number of classes

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # reshape due to cross-entropy
            targets = targets.view(B*T) # reshape due to cross-entropy
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx: torch.Tensor, max_new_tokens: int) -> torch.Tensor: # to generate new tokens
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens): # max_new_tokens is the maximum number of tokens to generate
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -self.block_size:]
            # get the predictions
            logits, _ = self(idx_cond) # loss is none here
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities for each class
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution using the probabilities
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [14]:
# bigram model training
batch_size = 32
block_size = 8
head_size = 16 # n_embd//n_heads
n_embd = 32
n_heads = 8
dropout_r = 0.1

eval_interval = 500
eval_iters = 200
max_iters = 5000
learning_rate = 1e-3
torch.manual_seed(1337)

def train(tokenizer = 'simple', vocab_size = vocab_size1):
    model = BigramLanguageModel(vocab_size=vocab_size, n_embd=n_embd, block_size=block_size, 
                                head_size = head_size, n_heads = n_heads, dropout_r = dropout_r)
    m = model.to(device)
    optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

    for iter in range(max_iters):
        if iter % eval_interval == 0:
            losses = estimate_loss(model, eval_iters = eval_iters, tokenizer = tokenizer, 
                                   batch_size = batch_size, block_size = block_size)
            print(f'Iter {iter}, train loss = {losses["train"]:.4f}, val loss = {losses["val"]:.4f}')

        xb, yb = get_batch('train', tokenizer, batch_size, block_size)
        logits, loss = m(xb, yb)
        optimizer.zero_grad(set_to_none=True) # If the user requests zero_grad(set_to_none=True) followed by a backward pass, .grads are guaranteed to be None for params that did not receive a gradient. 
        loss.backward()
        optimizer.step()
        
    return m

In [15]:
m1 = train(tokenizer = 'simple', vocab_size = vocab_size1)
# check the generated text after training
example_x = torch.zeros((1, 1), dtype=torch.long, device = device)
print(decoder1(m1.generate(idx = example_x, max_new_tokens=100)[0].tolist()))

Iter 0, train loss = 4.3966, val loss = 4.3933
Iter 500, train loss = 2.3737, val loss = 2.3804
Iter 1000, train loss = 2.2312, val loss = 2.2625
Iter 1500, train loss = 2.1477, val loss = 2.1814
Iter 2000, train loss = 2.0971, val loss = 2.1596
Iter 2500, train loss = 2.0547, val loss = 2.1229
Iter 3000, train loss = 2.0090, val loss = 2.0948
Iter 3500, train loss = 1.9883, val loss = 2.0825
Iter 4000, train loss = 1.9755, val loss = 2.0776
Iter 4500, train loss = 1.9389, val loss = 2.0511

Rur parten: I ming yu; our anstogeraty,
To notsme pralce to hat love
shall my soselder I at brity to


In [20]:
m2 = train(tokenizer = 'sentencepiece', vocab_size = vocab_size2)
# check the generated text after training
example_x = torch.zeros((1, 1), dtype=torch.long, device = device)
print(decoder2(m2.generate(idx = example_x, max_new_tokens=100)[0].tolist()))

Iter 0, train loss = 7.0807, val loss = 7.0758
Iter 500, train loss = 5.4188, val loss = 5.4972
Iter 1000, train loss = 4.8801, val loss = 5.0981
Iter 1500, train loss = 4.6675, val loss = 4.9566
Iter 2000, train loss = 4.5488, val loss = 4.8549
Iter 2500, train loss = 4.4249, val loss = 4.8327
Iter 3000, train loss = 4.3620, val loss = 4.7415
Iter 3500, train loss = 4.3085, val loss = 4.7563
Iter 4000, train loss = 4.2745, val loss = 4.7262
Iter 4500, train loss = 4.1842, val loss = 4.6817
 ⁇ ABETH: Ho this my face I fear of this prel, splest, Retitter you have muther, Or when I nacrifment son 'Tis would make. LORD VOLUMNIA: Hape from murder! how bring the father's pats will thates: To hear Fin away being Jight 'Tis not nightove trouarld'd which the humblur


In [21]:
m3 = train(tokenizer = 'tiktoken', vocab_size = vocab_size3)
# check the generated text after training
example_x = torch.zeros((1, 1), dtype=torch.long, device = device)
print(decoder3(m3.generate(idx = example_x, max_new_tokens=100)[0].tolist()))

Iter 0, train loss = 11.6907, val loss = 11.6948
Iter 500, train loss = 6.6760, val loss = 7.0683
Iter 1000, train loss = 5.9611, val loss = 6.6715
Iter 1500, train loss = 5.6093, val loss = 6.4514
Iter 2000, train loss = 5.4087, val loss = 6.3321
Iter 2500, train loss = 5.2206, val loss = 6.3148
Iter 3000, train loss = 5.0945, val loss = 6.2945
Iter 3500, train loss = 5.0143, val loss = 6.3112
Iter 4000, train loss = 4.9372, val loss = 6.2748
Iter 4500, train loss = 4.8775, val loss = 6.2482
! O, up the infinite, you, let friends.
Well, greatTriState condemn, you know.

ANGELO:
Hardam, my lord, only outemon! yet: painted
A faithful royal prayer'd'd and co spider to no mortal, that, suitthacks,ft of all to himself.

DUKE OF YORK:
Hear, thanks, Marci shall
Withqu crown: successful, with him.

Second MurderCIUS:
O the rest gone as word,
Of craft no


In [24]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Total number of parameters: {count_parameters(m1)}')
print(f'Total number of parameters: {count_parameters(m2)}')
print(f'Total number of parameters: {count_parameters(m3)}')

Total number of parameters: 153921
Total number of parameters: 214696
Total number of parameters: 6667701


In [ ]:
# try wordpiece tokenizer: pip install word-piece-tokenizer
# from word_piece_tokenizer import WordPieceTokenizer
# tokenizer = WordPieceTokenizer()
# ids = tokenizer.tokenize('reading a storybook!')
# tokens = tokenizer.convert_ids_to_tokens(ids)

In [ ]:
# losses may not be comparable due to different vocabulary sizes

# larger vocabulary sizes also make training slower: The vocabulary size can significantly impact the training speed of a Transformer model in several ways:
# - Embedding Layer Size: size of embedding matrix grows
# - Softmax Computation: larger vocabulary means more computations per token
# - Attention Mechanism: the overall computation per token includes the embedding lookups and softmax operations --> slower with larger vocab size

# - mitigation: Subword Tokenization: 
# - Methods like Byte Pair Encoding (BPE) or WordPiece can reduce the effective vocabulary size by breaking words into smaller subword units. This helps in balancing between expressiveness and computational efficiency.
# - Sampling-Based Softmax: Negative Sampling or Hierarchical Softmax can reduce the computational cost of the softmax function by approximating the full softmax
# - Mixed-Precision Training: Using lower precision (like float16 instead of float32) for embeddings and other parameters can reduce memory usage and improve computational speed without significantly affecting model performance.